# On crée le modèle

Exemple: une couche de 10 neurones et une couche de 5 :

In [ ]:
hidden_size = [10, 5]

ou autre exemple avec une couche de 3 neurones qui est siffisant pour l'environnement 1 :

In [1]:
hidden_size = [3]

In [102]:
# Importe le modèle préparé par PE
from model.DeepNN import DeepNetwork

# Entrée: 1 action 
# Sortie: 2 probabilités (une pour chaque outcome)
model_ML = DeepNetwork(hidden_size=hidden_size, input_size=1, output_size=2)
model_ML.to("cpu")

liste hidden init [3]


DeepNetwork(
  (fc1): Linear(in_features=1, out_features=3, bias=True)
  (hidden_layers): ModuleList()
  (fc4): Linear(in_features=3, out_features=2, bias=True)
)

# On tokenize le vocabulaire

In [4]:
# Import le tokenizer préparé par PE
from model.Tokenizer import SimpleTokenizerV1
from outil import create_dico_numerate_word

# Notre vocabulaire:
all_word = create_dico_numerate_word(['x', 'y', 'a', 'b'])
# Voici all_word : {'x': 0, 'y': 1, 'a': 2, 'b': 3}
print("Vocabulaire:", all_word)
# Nous n'avons pas besoin d'un tokenzier complexe, 
# il va avoir le même rôle qu'un dictionnaire
tokenizer = SimpleTokenizerV1(all_word)
# Maintenant pour encoder une action ou un outcome nous pouvons faire :
print('Test encode a:', tokenizer.encode('a'))
# et pour decoder :
print('Test decode l:', tokenizer.decode(1))

Vocabulaire: {'x': 0, 'y': 1, 'a': 2, 'b': 3}
Test encode a: 2
Test decode l: y


# On définit les paramètres du modèle

Le `lr` et le `weight_decay` sont des paramètres très importants qui vont permettre de corriger plus ou moins vite le modèle. Une correction rapide est moins précise.


In [5]:
lr = 1e-1
weight_decay = 1e-2

In [6]:
# On importe les outils fournis par torch
import torch
import torch.nn as nn

# Le modèle a besoin d'un optimizer pour corriger les poids.
optimizer = torch.optim.Adam(model_ML.parameters(), lr=lr, weight_decay=weight_decay)

# Nous avons aussi besoin d'une fonction de loss, par exemple la CrossEntropy
# Au vue de la simplicité de prédiction le type de loss importe peu
loss_func = nn.CrossEntropyLoss()

# On crée un dataset d'évaluation

l'action `a` renvoie l'outcome `x`. L'action `b` renvoie l'outcome `y`.

In [7]:
tokenized_actions = [[tokenizer.encode('a')], [tokenizer.encode('b')]]
tokenized_outomes = [tokenizer.encode('x'), tokenizer.encode('y')]

# Convertit en tensors pour torch
x = torch.tensor(tokenized_actions, dtype=torch.float).to("cpu")
y = torch.tensor(tokenized_outomes, dtype=torch.long).to("cpu")

# Notre dataset de test :
print("x (tokenized actions):\n", x)
print("y (tokenized outcomes):\n", y)

x (tokenized actions):
 tensor([[2.],
        [3.]])
y (tokenized outcomes):
 tensor([0, 1])


On crée un loader pour charger le dataset d'évaluation

In [8]:
# Nous utilisons un dataLoader de pytorch
# Le batch_size n'a pas d'importance de même que le shuffle
validate_loader = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(x, y),
    batch_size=32, shuffle=False
)

# La fonction fit()

In [11]:
# Importe la fonction train préparée par PE
from model.DeepNN import train

def fit(action, outcome):
    action = tokenizer.encode(action)
    action = torch.tensor([action], dtype=torch.float).to("cpu")
    outcome = tokenizer.encode(outcome)
    outcome = torch.tensor(outcome, dtype=torch.long).to("cpu")
    outcome = torch.nn.functional.one_hot(outcome, num_classes=2).to(torch.float)
    # Qu'est ce que ca fait ?
    model_ML.train()

    train(model=model_ML, 
          train_data=[(action, outcome)],
          optimizer=optimizer,
          loss_func=loss_func,
          nb_epochs=1,
          validate_loader=None, # validate_loader,
          print_=True)

On entraine le modèle dans en environnement ou action `a` produit outcome `x` et action `b` produit outcome `y`.

Utiliser `Ctrl` + `Enter` pour l'exécuter plusieurs fois et voir la loss diminuer à chaque appel.

In [95]:
fit('a', 'x')
fit('b', 'y')

Epoch 1/1,, Loss: 0.1508
Epoch 1/1,, Loss: 0.1062


# La fonction predict()

In [62]:
def predict(action):
    action = tokenizer.encode(action)
    action = torch.tensor([action], dtype=torch.float).to("cpu")
    model_ML.eval() # On peut le passer en mode evaluation
    prediction_tensor = model_ML(action)
    print(prediction_tensor)
    predicted_outcome_token = torch.argmax(prediction_tensor, dim=0).item()
    return tokenizer.decode(predicted_outcome_token)

In [100]:
predicted_outcome = predict('a')
print("Predication pour a:", predicted_outcome)
predicted_outcome = predict('b')
print("Predication pour b:", predicted_outcome)

tensor([ 0.5221, -1.3315], grad_fn=<ViewBackward0>)
Predication pour a: x
tensor([-1.5422,  0.7371], grad_fn=<ViewBackward0>)
Predication pour b: y


Si le modèle a été suffisement entrainé, il prédit correctement que action `a` produit outcome `x` et `b` produit `y`.